In [1]:
import pandas as pd
import time
import re
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_absolute_error
from tqdm import tqdm
import random
import ollama
from google.colab import drive

In [2]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/DT_SensorData/Occupancy_Estimation.csv')


Mounted at /content/drive


In [3]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,209 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,297 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InR

In [4]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama","serve"])

thread= threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [5]:
!ollama pull llama3.1:8b
!ollama pull zephyr
!ollama pull deepseek-r1:7b
#!ollama pull llama3.1:70b

In [6]:
def prepare_data(file_path='/content/drive/MyDrive/DT_SensorData/Occupancy_Estimation.csv'):
    """Carica e preprocessa il dataset."""
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"ERRORE: File '{file_path}' non trovato.")
        return None

    df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df = df.sort_values('Timestamp').set_index('Timestamp')

    df['Temperature'] = df[['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp']].mean(axis=1)
    df['Light'] = df[['S1_Light', 'S2_Light', 'S3_Light', 'S4_Light']].mean(axis=1)
    df['Sound'] = df[['S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound']].mean(axis=1)
    df['CO2'] = df['S5_CO2']
    features_df = df[['Temperature', 'Light', 'Sound', 'CO2', 'Room_Occupancy_Count']]

    features_df['Occupancy_Change'] = features_df['Room_Occupancy_Count'].diff().apply(
        lambda x: 'INCREASE' if x > 0 else ('DECREASE' if x < 0 else 'STABLE')
    )

    features_to_predict = ['Temperature', 'Light', 'CO2', 'Sound']
    for feature in features_to_predict:
        features_df[f'next_{feature}'] = features_df[feature].shift(-1)

    return features_df.dropna()

def create_sliding_windows(data, window_size):
    """Crea finestre mobili dal dataset."""
    windows_text, future_values, occupancy_outcomes = [], [], []
    cols_for_text = ['Temperature', 'Light', 'CO2', 'Sound', 'Room_Occupancy_Count']
    for i in range(len(data) - window_size):
        window_df = data[cols_for_text].iloc[i : i + window_size]
        text_lines = []
        for index, row in window_df.iterrows():
            line = (f"Time: {index.strftime('%H:%M:%S')}, Temp: {row['Temperature']:.2f}C, Light: {row['Light']:.1f} Lux, "
                    f"CO2: {row['CO2']:.1f} ppm, Sound: {row['Sound']:.2f}, Occupancy: {int(row['Room_Occupancy_Count'])}")
            text_lines.append(line)
        windows_text.append("\n".join(text_lines))
        target_row = data.iloc[i + window_size]
        future_values.append(target_row[[f'next_{f}' for f in ['Temperature', 'Light', 'CO2', 'Sound']]].to_dict())
        occupancy_outcomes.append(target_row['Occupancy_Change'])
    return windows_text, future_values, occupancy_outcomes

In [7]:
def create_forecast_prompt(data_window):
    system_message = (
        "You are a sophisticated time-series analysis AI. Your task is to analyze environmental sensor data. "
        "You MUST follow the output format EXACTLY. Your response MUST start with 'FORECAST:'.\n\n"
        "1. **FORECAST**: Predict the sensor values for the next timestamp.\n"
        "2. **ANALYSIS**: Write a brief narrative analysis explaining the trend and what it means.\n"
        "3. **PREDICTION**: Output ONLY ONE of the three following words: INCREASE, DECREASE, or STABLE.\n\n"
        "Respond ONLY in the following structured format:\n\n"
        "FORECAST:\n- Temperature: [value]°C\n- Light: [value] Lux\n- CO2: [value] ppm\n- Sound: [value]\n\n"
        "ANALYSIS:\n[Your one or two-sentence narrative analysis here.]\n\n"
        "PREDICTION: [INCREASE / DECREASE / STABLE]"
    )
    user_prompt = (
        "Analyze the following sequence of sensor readings and provide the forecast, analysis, and final prediction in the structured format.\n\n"
        f"--- PAST DATA ---\n{data_window}\n--- END OF DATA ---"
    )
    return system_message, user_prompt

def create_few_shot_prompt(data_window):
    system_message = (
        "You are a sophisticated time-series analysis AI. Follow the examples EXACTLY. Your response MUST start with 'FORECAST:'. "
        "Do NOT add any extra text, comments, or markdown formatting outside of the required structure."
    )
    example_1_data = "Time: 08:32:17, Temp: 24.65C, Light: 480.5 Lux, CO2: 485.8 ppm, Sound: 0.25, Occupancy: 1"
    example_1_output = ("FORECAST:\n- Temperature: 24.70°C\n- Light: 480.5 Lux\n- CO2: 510.0 ppm\n- Sound: 0.15\n\n"
                      "ANALYSIS:\nThe dramatic increase in light and rising CO2 suggest a person has entered.\n\n"
                      "PREDICTION: STABLE")
    example_2_data = "Time: 17:47:07, Temp: 25.00C, Light: 0.0 Lux, CO2: 720.5 ppm, Sound: 0.04, Occupancy: 0"
    example_2_output = ("FORECAST:\n- Temperature: 24.95°C\n- Light: 0.0 Lux\n- CO2: 690.0 ppm\n- Sound: 0.02\n\n"
                      "ANALYSIS:\nThe light is off and CO2 is declining, indicating the room is now empty.\n\n"
                      "PREDICTION: STABLE")
    user_prompt = (
        "Here are some examples of the task.\n\n"
        f"### EXAMPLE 1 ###\n--- PAST DATA ---\n{example_1_data}\n--- END OF DATA ---\n\n{example_1_output}\n\n"
        f"### EXAMPLE 2 ###\n--- PAST DATA ---\n{example_2_data}\n--- END OF DATA ---\n\n{example_2_output}\n\n"
        "--- END OF EXAMPLES ---\n\n"
        "### NEW TASK ###\n"
        "Now, perform the task for the following new data.\n\n"
        f"--- PAST DATA ---\n{data_window}\n--- END OF DATA ---"
    )
    return system_message, user_prompt

PROMPT_FUNCTIONS = {'zero_shot': create_forecast_prompt, 'few_shot': create_few_shot_prompt}

def get_structured_prediction_from_llm(model_name, system_prompt, user_prompt):
    # ... (Il contenuto di questa funzione rimane invariato)
    start_time = time.time()
    try:
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'system', 'content': system_prompt}, {'role': 'user', 'content': user_prompt}],
            options={'temperature': 0.1}
        )
        content = response['message']['content']
        execution_time = time.time() - start_time

        content = re.sub(r'<think>.*?</think>', '', content, flags=re.DOTALL)
        content = re.sub(r'#+\s*(FORECAST|ANALYSIS|PREDICTION)\s*#+', r'\1:', content, flags=re.IGNORECASE)
        content = re.sub(r'-+\s*(FORECAST|ANALYSIS|PREDICTION)\s*-+', r'\1:', content, flags=re.IGNORECASE)

        forecast_match = re.search(r"FORECAST:\s*(.*?)\s*ANALYSIS:", content, re.DOTALL | re.IGNORECASE)
        analysis_match = re.search(r"ANALYSIS:\s*(.*?)\s*PREDICTION:", content, re.DOTALL | re.IGNORECASE)
        prediction_match = re.search(r"PREDICTION:\s*(INCREASE|DECREASE|STABLE)", content, re.IGNORECASE)

        analysis_text = analysis_match.group(1).strip() if analysis_match else "PARSE_ERROR"
        prediction_text = prediction_match.group(1).upper() if prediction_match else "PARSE_ERROR"
        forecast_text = forecast_match.group(1).strip() if forecast_match else "PARSE_ERROR"

        forecast_values = {}
        if forecast_text != "PARSE_ERROR":
            temp_val = re.search(r"Temperature:\s*([0-9\.\-]+)", forecast_text)
            light_val = re.search(r"Light:\s*([0-9\.\-]+)", forecast_text)
            co2_val = re.search(r"CO2:\s*([0-9\.\-]+)", forecast_text)
            sound_val = re.search(r"Sound:\s*([0-9\.\-]+)", forecast_text)
            forecast_values['Temperature'] = float(temp_val.group(1)) if temp_val else -1.0
            forecast_values['Light'] = float(light_val.group(1)) if light_val else -1.0
            forecast_values['CO2'] = float(co2_val.group(1)) if co2_val else -1.0
            forecast_values['Sound'] = float(sound_val.group(1)) if sound_val else -1.0

        return forecast_values, analysis_text, prediction_text, execution_time

    except Exception as e:
        print(f"\n--- ERRORE durante la chiamata a {model_name}: {e} ---")
        return {}, "API_ERROR", "API_ERROR", time.time() - start_time


In [8]:
def create_judge_prompt(input_data, generated_analysis):
    # ... (Il contenuto di questa funzione rimane invariato)
    system_message = (
        "You are an expert evaluator of AI-generated text. Your task is to assess the quality of a textual analysis "
        "generated by another language model based on a time-series of sensor data. "
        "You must evaluate the analysis based on the provided criteria and respond ONLY with a valid JSON object."
    )
    user_prompt = (
        "Please evaluate the 'Generated Analysis' below based on the provided 'Sensor Data'.\n\n"
        "### Sensor Data (Input) ###\n"
        f"{input_data}\n\n"
        "### Generated Analysis (to be evaluated) ###\n"
        f'"{generated_analysis}"\n\n'
        "### Evaluation Criteria ###\n"
        "1.  **Relevance (score 1-5):** How well is the analysis grounded in the provided sensor data?\n"
        "2.  **Clarity (score 1-5):** Is the analysis clear and well-written?\n"
        "3.  **Logical Coherence (score 1-5):** Is the reasoning sound?\n\n"
        "### Your Task ###\n"
        "Provide your evaluation ONLY in the following JSON format. Do not add any text before or after the JSON object.\n"
        '{"relevance_score": [score], "clarity_score": [score], "coherence_score": [score], "justification": "[your brief justification]"}'
    )
    return system_message, user_prompt

def get_evaluation_from_judge(input_data, generated_analysis):
    # ... (Il contenuto di questa funzione rimane invariato)
    try:
        system_p, user_p = create_judge_prompt(input_data, generated_analysis)
        response = ollama.chat(
            model=JUDGE_MODEL,
            messages=[{'role': 'system', 'content': system_p}, {'role': 'user', 'content': user_p}],
            format="json"
        )
        evaluation_json = json.loads(response['message']['content'])
        return evaluation_json
    except Exception as e:
        return {"relevance_score": -1, "clarity_score": -1, "coherence_score": -1, "justification": f"EVALUATION_ERROR: {str(e)}"}


In [9]:
# --- Parametri per l'Esperimento Principale ---
MODELS_TO_TEST = ['llama3.1:8b', 'deepseek-r1:7b', 'zephyr:latest']
WINDOW_SIZES_TO_TEST = [15]
NUM_SAMPLES_TO_TEST = 400  # Numero di finestre casuali da testare per ogni configurazione
RESULTS_OUTPUT_FILE = '/content/drive/MyDrive/DT_SensorData/llm_forecast_results.csv'
SUMMARY_OUTPUT_FILE = '/content/drive/MyDrive/DT_SensorData/performance_summary.csv'
PROMPT_FUNCTIONS = {'zero_shot': create_forecast_prompt, 'few_shot': create_few_shot_prompt}

# --- Parametri per la Valutazione Qualitativa ---
JUDGE_MODEL = 'llama3.1:70b' # Il modello che farà da giudice
EVALUATION_OUTPUT_FILE = '/content/drive/MyDrive/DT_SensorData/qualitative_evaluation_sample.csv'

NUM_SAMPLES_TO_EVALUATE = 50

In [10]:
def run_experiments():
    """Esegue l'esperimento principale e la valutazione quantitativa."""
    features_df = prepare_data()
    if features_df is None: return

    result_columns = [
        'model', 'window_size', 'prompt_type', 'window_index', 'input_data',
        'true_occupancy_label', 'predicted_occupancy_label', 'true_sensor_values',
        'predicted_forecast_values', 'predicted_analysis', 'execution_time'
    ]
    try:
        pd.read_csv(RESULTS_OUTPUT_FILE)
    except FileNotFoundError:
        pd.DataFrame(columns=result_columns).to_csv(RESULTS_OUTPUT_FILE, index=False)

    print(f"🚀 Inizio esperimenti. Verranno testate {NUM_SAMPLES_TO_TEST} finestre casuali per configurazione.")

    for model in MODELS_TO_TEST:
        for ws in WINDOW_SIZES_TO_TEST:
            print(f"\nAnalizzando: Modello={model}, Window Size={ws}")
            full_text_windows, full_future_values, full_occupancy_labels = create_sliding_windows(features_df, ws)

            all_indices = list(range(len(full_text_windows)))
            if NUM_SAMPLES_TO_TEST >= len(all_indices):
                sampled_indices = all_indices
            else:
                random.seed(42)
                sampled_indices = random.sample(all_indices, NUM_SAMPLES_TO_TEST)
            print(f"  -> Selezionati {len(sampled_indices)} indici casuali per il test.")

            for p_name, p_func in PROMPT_FUNCTIONS.items():
                progress_bar = tqdm(sampled_indices, desc=f"  Prompt: {p_name}")
                for original_index in progress_bar:
                    window_data = full_text_windows[original_index]
                    true_occupancy_label = full_occupancy_labels[original_index]
                    true_sensor_values = full_future_values[original_index]

                    system_p, user_p = p_func(window_data)
                    pred_forecast, pred_analysis, pred_label, ex_time = get_structured_prediction_from_llm(model, system_p, user_p)

                    single_result = {
                        'model': model, 'window_size': ws, 'prompt_type': p_name, 'window_index': original_index,
                        'input_data': window_data, 'true_occupancy_label': true_occupancy_label,
                        'predicted_occupancy_label': pred_label, 'true_sensor_values': json.dumps(true_sensor_values),
                        'predicted_forecast_values': json.dumps(pred_forecast), 'predicted_analysis': pred_analysis,
                        'execution_time': ex_time
                    }

                    result_df_row = pd.DataFrame([single_result])
                    result_df_row.to_csv(RESULTS_OUTPUT_FILE, mode='a', header=False, index=False)

    print(f"\n✅ Esperimento completato! Tutti i risultati sono stati salvati in '{RESULTS_OUTPUT_FILE}'")

     # --- INIZIO BLOCCO MANCANTE: VALUTAZIONE QUANTITATIVA ---
    print(f"\n📊 Avvio valutazione quantitativa da '{RESULTS_OUTPUT_FILE}'...")

    # Ricarica il dataframe completo appena salvato
    results_df = pd.read_csv(RESULTS_OUTPUT_FILE)

    valid_results = results_df[~results_df['predicted_occupancy_label'].isin(['PARSE_ERROR', 'API_ERROR'])].copy()
    grouped_analysis = valid_results.groupby(['model', 'window_size', 'prompt_type'])
    summary_metrics = []

    for name, group in grouped_analysis:
        y_true_class = group['true_occupancy_label']
        y_pred_class = group['predicted_occupancy_label']

        classification_metrics = {
            'accuracy': accuracy_score(y_true_class, y_pred_class),
            'f1_score_weighted': f1_score(y_true_class, y_pred_class, average='weighted', zero_division=0)
        }

        predicted_forecasts = [json.loads(v) for v in group['predicted_forecast_values'] if '"Temperature"' in v]
        actual_values = [json.loads(v) for v in group['true_sensor_values']]

        mae_scores = {}
        features_to_predict = ['Temperature', 'Light', 'CO2', 'Sound']
        for feature in features_to_predict:
            y_pred_feature = [d.get(feature, -1.0) for d in predicted_forecasts]
            y_true_feature = [d.get(f'next_{feature}', -1.0) for d in actual_values]
            valid_pairs = [(t, p) for t, p in zip(y_true_feature, y_pred_feature) if p is not None and t is not None and p != -1.0 and t != -1.0]
            if valid_pairs:
                y_true_valid, y_pred_valid = zip(*valid_pairs)
                mae_scores[f'MAE_{feature}'] = mean_absolute_error(y_true_valid, y_pred_valid)
            else:
                mae_scores[f'MAE_{feature}'] = np.nan

        current_summary = {'model': name[0], 'window_size': name[1], 'prompt_type': name[2]}
        current_summary.update(classification_metrics)
        current_summary.update(mae_scores)
        summary_metrics.append(current_summary)

    summary_df = pd.DataFrame(summary_metrics)
    summary_df.to_csv(SUMMARY_OUTPUT_FILE, index=False)

    print(f"\n--- Riepilogo Performance Quantitative ---")
    print(summary_df.round(3))
    print(f"\n✅ Valutazione quantitativa completata! Riepilogo salvato in '{SUMMARY_OUTPUT_FILE}'")
    # --- FINE BLOCCO MANCANTE ---


def run_qualitative_evaluation():
    """Esegue la valutazione qualitativa usando l'LLM-as-a-Judge."""
    print(f"\n\n📊 Avvio valutazione qualitativa con il giudice: {JUDGE_MODEL}")

    NUM_SAMPLES_PER_GROUP = 10

    try:
        results_df = pd.read_csv(RESULTS_OUTPUT_FILE)
    except FileNotFoundError:
        print(f"ERRORE: File '{RESULTS_OUTPUT_FILE}' non trovato. Esegui prima 'run_experiments()'.")
        return

    valid_results = results_df[~results_df['predicted_analysis'].isin(['PARSE_ERROR', 'API_ERROR', ''])].dropna(subset=['predicted_analysis']).copy()

    # --- INIZIO MODIFICA ---
    try:
        sample_df = valid_results.groupby(
            ['model', 'window_size', 'prompt_type'], group_keys=False
        ).apply(
            lambda x: x.sample(min(len(x), NUM_SAMPLES_PER_GROUP), random_state=42)
        )
    except Exception as e:
        print(f"Errore durante il campionamento bilanciato: {e}")
        # Fallback al campionamento semplice se il raggruppamento fallisce
        sample_df = valid_results.sample(n=min(len(valid_results), NUM_SAMPLES_PER_GROUP*6), random_state=42)


    print(f"Campionamento bilanciato eseguito. Verranno valutati {len(sample_df)} campioni totali ({NUM_SAMPLES_PER_GROUP} per ogni configurazione).")

    if sample_df.empty:
        print("Nessuna analisi valida trovata per la valutazione.")
        return

    all_evaluations = []
    progress_bar = tqdm(sample_df.iterrows(), total=len(sample_df), desc="Valutando i campioni")
    for index, row in progress_bar:
        evaluation = get_evaluation_from_judge(row['input_data'], row['predicted_analysis'])
        evaluation_result = row.to_dict()
        evaluation_result.update(evaluation)
        all_evaluations.append(evaluation_result)

    evaluation_df = pd.DataFrame(all_evaluations)
    evaluation_df.to_csv(EVALUATION_OUTPUT_FILE, index=False)
    print(f"\n✅ Valutazione qualitativa completata! Risultati salvati in '{EVALUATION_OUTPUT_FILE}'")

    print("\n--- Riepilogo Punteggi Medi di Valutazione Qualitativa ---")
    final_scores = evaluation_df[evaluation_df['relevance_score'] != -1]
    if not final_scores.empty:
        summary = final_scores.groupby(['model', 'prompt_type'])[['relevance_score', 'clarity_score', 'coherence_score']].mean().round(2)
        print(summary)
    else:
        print("Nessuna valutazione qualitativa completata con successo da mostrare.")

In [ ]:
run_experiments()


/tmp/ipython-input-3631408001.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df['Occupancy_Change'] = features_df['Room_Occupancy_Count'].diff().apply(
/tmp/ipython-input-3631408001.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df[f'next_{feature}'] = features_df[feature].shift(-1)
/tmp/ipython-input-3631408001.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

🚀 Inizio esperimenti. Verranno testate 400 finestre casuali per configurazione.

Analizzando: Modello=llama3.1:8b, Window Size=15
  -> Selezionati 400 indici casuali per il test.


  Prompt: few_shot: 100%|██████████| 400/400 [07:19<00:00,  1.10s/it]



Analizzando: Modello=deepseek-r1:7b, Window Size=15
  -> Selezionati 400 indici casuali per il test.


  Prompt: few_shot: 100%|██████████| 400/400 [39:24<00:00,  5.91s/it]



Analizzando: Modello=zephyr:latest, Window Size=15
  -> Selezionati 400 indici casuali per il test.


  Prompt: few_shot:  31%|███       | 124/400 [1:20:34<4:12:46, 54.95s/it]

In [ ]:
run_qualitative_evaluation()


/tmp/ipython-input-1410377323.py:120: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(




📊 Avvio valutazione qualitativa con il giudice: llama3.1:70b
Campionamento bilanciato eseguito. Verranno valutati 100 campioni totali (10 per ogni configurazione).


Valutando i campioni: 100%|██████████| 100/100 [28:19<00:00, 17.00s/it]



✅ Valutazione qualitativa completata! Risultati salvati in '/content/drive/MyDrive/DT_SensorData/qualitative_evaluation_sample.csv'

--- Riepilogo Punteggi Medi di Valutazione Qualitativa ---
                            relevance_score  clarity_score  coherence_score
model          prompt_type                                                 
deepseek-r1:7b few_shot                4.40           5.00             4.40
               zero_shot               4.50           5.00             4.30
llama3.1:8b    few_shot                4.30           5.00             4.10
               zero_shot               3.90           4.95             3.80
zephyr:latest  few_shot                4.30           4.95             4.10
               zero_shot               4.05           4.90             3.95
